In [ ]:
import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

from model import build_encoder, build_autoencoder
from data import example_to_tensor, normalize, add_channel_axis, train_test_split
from utils import duplicate_iterator, plot_slice, plot_animated_volume
from config import allocate_gpu_memory_only_when_needed, data_root_dir, seed

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

In [ ]:
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
#
# policy = mixed_precision.Policy("mixed_float16")
##policy = mixed_precision.Policy("float32")
# mixed_precision.set_policy(policy)
# print("Compute dtype: %s" % policy.compute_dtype)
# print("Variable dtype: %s" % policy.variable_dtype)

In [ ]:
downscaling = 2
if downscaling == 4:
    input_shape = (24, 128, 128, 1)
    neg_tfrecord_glob = "covid-neg-0.25/*.tfrecord"
    pos_tfrecord_glob = "covid-pos-0.25/*.tfrecord"
elif downscaling == 2:
    input_shape = (48, 256, 256, 1)
    # neg_tfrecord_glob = "covid-neg-0.25/*.tfrecord"
    # pos_tfrecord_glob = "covid-pos-0.25/*.tfrecord"
    neg_tfrecord_glob = "CT-0-0.5/*.tfrecord"
    pos_tfrecord_glob = "CT-[1-4]-0.5/*.tfrecord"
elif downscaling == 1:
    input_shape = (96, 512, 512, 1)
    neg_tfrecord_glob = "covid-neg-0.25/*.tfrecord"
    pos_tfrecord_glob = "covid-pos-0.25/*.tfrecord"
else:
    raise RuntimeError("Downscaling not supported")

encoder_num_filters = [32, 64, 128]
epochs = 1000
patience = 20
batch_size = 8
learning_rate = 0.0001
val_perc = 0.1  # percentage from the already splitted training test
test_perc = 0.1

In [ ]:
neg_tfrecord_fnames = [str(p) for p in Path(data_root_dir).glob(neg_tfrecord_glob)]
neg_x = (
    tf.data.TFRecordDataset(neg_tfrecord_fnames)
    .map(example_to_tensor, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(add_channel_axis, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)
# num_neg = sum(1 for _ in neg_x)
num_neg = 254
print(f"Number of negative samples: {num_neg}")
neg_x

In [ ]:
pos_tfrecord_fnames = [str(p) for p in Path(data_root_dir).glob(pos_tfrecord_glob)]
pos_x = (
    tf.data.TFRecordDataset(pos_tfrecord_fnames)
    .map(example_to_tensor, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(add_channel_axis, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)
# num_pos = sum(1 for _ in pos_x)
num_pos = 856
print(f"Number of positive samples: {num_pos}")
pos_x

In [ ]:
neg_y = tf.data.Dataset.from_tensors(tf.constant([0], dtype=tf.int8)).repeat(num_neg)
neg_dataset = tf.data.Dataset.zip((neg_x, neg_y))
neg_dataset

In [ ]:
pos_y = tf.data.Dataset.from_tensors(tf.constant([1], dtype=tf.int8)).repeat(num_pos)
pos_dataset = tf.data.Dataset.zip((pos_x, pos_y))
pos_dataset

In [ ]:
dataset = neg_dataset.concatenate(pos_dataset)
dataset, test_dataset = train_test_split(
    dataset,
    test_perc=test_perc,
    cardinality=None,
    seed=seed,
)
test_dataset = test_dataset.padded_batch(1, (input_shape, (1,)))
train_dataset, val_dataset = train_test_split(
    dataset,
    test_perc=val_perc,
    cardinality=None,
    seed=seed,
)
val_dataset = (
    val_dataset.padded_batch(batch_size, (input_shape, (1,)), drop_remainder=True)
    .cache()
    .prefetch(tf.data.experimental.AUTOTUNE)
)
train_dataset = (
    train_dataset.padded_batch(batch_size, (input_shape, (1,)), drop_remainder=True)
    .cache()  # must be called before shuffle
    .shuffle(buffer_size=64, reshuffle_each_iteration=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
train_dataset

In [ ]:
from collections import Counter

print(Counter(label.numpy()[0] for _, label in train_dataset.unbatch()))
print(Counter(label.numpy()[0] for _, label in val_dataset.unbatch()))
print(Counter(label.numpy()[0] for _, label in test_dataset.unbatch()))

In [ ]:
pretrained = False
if pretrained:
    encoder = keras.models.load_model(
        "models/autoencoder-20201023-172335.h5"
    ).get_layer("encoder")
    encoder.trainable = False  # free the pre-trained encoder
else:
    encoder = build_encoder(input_shape)
encoder.summary()

In [ ]:
cnn = keras.Sequential(
    [
        encoder,
        keras.layers.Flatten(),
        keras.layers.Dense(
            512,
            kernel_initializer="lecun_normal",
            bias_initializer="lecun_normal",
            activation="selu",
        ),
        keras.layers.AlphaDropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ],
    name="cnn",
)
cnn.summary()

In [ ]:
cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[
        keras.metrics.TruePositives(name="tp"),
        keras.metrics.FalsePositives(name="fp"),
        keras.metrics.TrueNegatives(name="tn"),
        keras.metrics.FalseNegatives(name="fn"),
        keras.metrics.BinaryAccuracy(name="accuracy"),
        keras.metrics.Precision(name="precision"),
        keras.metrics.Recall(name="recall"),
        keras.metrics.AUC(name="auc"),
    ],
)
monitor_metric = "val_accuracy"

start_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
best_checkpoint = f"models/{'pretrained-' if pretrained else ''}{start_time}.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    best_checkpoint, monitor=monitor_metric, mode="max", verbose=1, save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(
    monitor=monitor_metric, patience=patience, mode="max"
)
log_dir = f"logs/{'pretrained-' if pretrained else ''}{start_time}"
file_writer = tf.summary.create_file_writer(log_dir)
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=f"logs/{'pretrained-' if pretrained else ''}{start_time}",
    histogram_freq=1,
    write_graph=False,
    profile_batch=0,
)
cnn.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb],
)
with file_writer.as_default():
    tf.summary.text(
        "Hyperparameters",
        f"{downscaling=}; "
        f"{encoder_num_filters=}; "
        f"{epochs=}; "
        f"{patience=}; "
        f"{batch_size=}; "
        f"{learning_rate=}; "
        f"{val_perc=}",
        step=0,
    )
cnn = keras.models.load_model(best_checkpoint)

In [ ]:
cnn = keras.models.load_model("models/pretrained-20201023-182623.h5")
cnn.evaluate(test_dataset, verbose=0, return_dict=True)

In [ ]:
cnn = keras.models.load_model("models/pretrained-20201021-230015.h5")
cnn.evaluate(test_dataset, verbose=0, return_dict=True)

In [ ]:
x, y = next(iter(test_dataset.skip(6)))
prediction = cnn(x, training=False)
print(f"real: {y.numpy()}, prediction: {prediction.numpy()}")
plot_animated_volume(x[0, :], fps=1)